# Lab 10 Tasks - Solution

In this we will look at using superised machine learning algorithms to predict music genres. Specficially, the objective here is to classify the genre of a song from Spotify, based on a range of associated audio features.

Each song in our dataset is described by a range of features:
- artist_name: Song artist 
- track_name: Song track name
- acousticness: Describes the likelihood that the song is purely acoustic
- danceability: Describes how suitable a track is for dancing based on a combination of elements including tempo, rhythm stability, beat strength, and overall regularity
- energy: A perceptual measure of intensity and activity. More energetic tracks feel fast, loud, and noisy
- instrumentalness: Indicates whether a song includes vocals or not
- liveness: Describes the likelihood that the song was recorded with a live audience.
- loudness: Overall loudness of a track in decibels (dB), averaged across the entire track
- speechiness:  Describes the likelihood that the song contains spoken words
- tempo: Estimated tempo of a track in beats per minute (BPM)
- valence: Tracks with high valence sound more positive (e.g. cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, angry)
- genre: Target label ('Pop' or 'Rock' in this case)

Original dataset source: 
https://www.kaggle.com/code/iqbalbasyar/spotify-genre-classification/data

Original Spotify documentation:
https://developer.spotify.com/discover/

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
# imports for plotting
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

## Task 1

Load the dataset from the file 'music.csv' and examine the number of songs having each target label.

In [ ]:
df = pd.read_csv("music.csv")
df.head()

In [ ]:
# check the number with each class label
df["genre"].value_counts()

Remove any non-numeric features from the dataset, and then separate out the features to use for classification from the target label information.

In [ ]:
# remove the non-numeric features
df.drop(["artist_name", "track_name"], axis=1, inplace=True)
df.head()

In [ ]:
# the target label
target = df["genre"]
# remove the target label, giving us just the numeric feratures for classification
data = df.drop("genre", axis=1).copy()

## Task 2

Generate a 60/40 random training and test split of the data. Based on this split, evaluate the accuracy and F1-score achieved by a KNN classifier (for *k=1* neighbour). 

In [ ]:
from sklearn.model_selection import train_test_split
# use 60% for training, 40% for testing
data_train, data_test, target_train, target_test = train_test_split(data, target, 
    test_size=0.4, random_state=1)
print("Training set has %d examples" % data_train.shape[0])
print("Test set has %d examples" % data_test.shape[0])

In [ ]:
# train the classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
m = knn.fit(data_train, target_train)
# make predictions for the test set
predicted = knn.predict(data_test)

In [ ]:
# evaluate accuracy
from sklearn.metrics import accuracy_score
acc = accuracy_score(target_test, predicted)
print("Accuracy=%.3f" % acc)
# evalaute F1 score
from sklearn.metrics import f1_score
print("F1(Rock) = %.3f" % f1_score(target_test, predicted, pos_label="Rock") )
print("F1(Pop) = %.3f" % f1_score(target_test, predicted, pos_label="Pop") )

Use a *confusion matrix* to illustrate where the errors lie with the classifier above.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# build the confusion matrix
cm = confusion_matrix(target_test, predicted)
# display it visually
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=knn.classes_)
disp.plot();

## Task 3



Use 5-fold cross-validation to evaluate the accuracy achieved by a KNN (*k=1*) classifier on the data.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
fold_scores = cross_val_score(knn, data, target, cv=5, scoring="accuracy")
mean_accuracies = {1: fold_scores.mean()}
print("KNN (k=1): Mean cross-validation accuracy = %.3f" % mean_accuracies[1])

Repeat the process above for different parameter values of *k*, from 2 to 10 neighbours. Generate a plot of the different accuracy values acheived for different values of *k*. Which value of *k* yields the highest accuracy?

In [ ]:
for k in range(2,11):
    # apply KNN for the specified value
    knn = KNeighborsClassifier(n_neighbors=k)
    fold_scores = cross_val_score(knn, data, target, cv=5, scoring="accuracy")
    mean_accuracies[k] = fold_scores.mean()

In [ ]:
# generate the plot
df_mean_accuracies = pd.Series(mean_accuracies)
ax = df_mean_accuracies.plot(figsize=(9.5,6), fontsize=14, lw=2, marker='o', color="darkorange")
ax.set_ylabel("Cross-validation Accuracy", fontsize=14)
ax.set_xlabel("Number of Neighbours (k)", fontsize=14)
ax.yaxis.grid()
ax.set_xlim(1, 10);

In [ ]:
# sort the values to see the parameter giving the highest accuracy
df_sorted = df_mean_accuracies.sort_values(ascending=False)
print(df_sorted)
# get the value of k which gives the highest accuracy
best_k = df_sorted.index[0]
print("Best value is k=%d" % best_k)

## Task 4

For certain datasets, classification may work better on a subset of features, rather than on the entire feature set (e.g. when noisy or misleading features are removed).

Using the KNN and the best value of *k* identified in Task 3, compare classification performance for the three feature subsets in the lists below. Which subset gives the highest accuracy?

In [ ]:
subset1 = ['danceability', 'energy', 'tempo', 'valence']
subset2 = ['acousticness', 'instrumentalness', 'liveness', 'speechiness']
subset3 = ['energy', 'tempo', 'valence', 'loudness']

In [ ]:
# try the first subset of features
data1 = data[subset1]
knn = KNeighborsClassifier(n_neighbors=best_k)
fold_scores = cross_val_score(knn, data1, target, cv=5, scoring="accuracy")
mean_acc1 = fold_scores.mean()
print("Subset 1 - KNN (k=%d): Mean cross-validation accuracy = %.3f" % (best_k, mean_acc1))

In [ ]:
# try the second subset of features
data2 = data[subset2]
knn = KNeighborsClassifier(n_neighbors=best_k)
fold_scores = cross_val_score(knn, data2, target, cv=5, scoring="accuracy")
mean_acc2 = fold_scores.mean()
print("Subset 2 - KNN (k=%d): Mean cross-validation accuracy = %.3f" % (best_k, mean_acc2))

In [ ]:
# try the third subset of features
data3 = data[subset3]
knn = KNeighborsClassifier(n_neighbors=best_k)
fold_scores = cross_val_score(knn, data3, target, cv=5, scoring="accuracy")
mean_acc3 = fold_scores.mean()
print("Subset 3 - KNN (k=%d): Mean cross-validation accuracy = %.3f" % (best_k, mean_acc3))